let's start by exploring our use cases

In [13]:
import pandas as pd

NEWLINE = '\n'
TAB = '\t'
db = pd.read_csv('../../resources/acme_security_tickets.csv')
print(f"db columns are:\n\t{(NEWLINE + TAB).join([str(c) for c in db.columns])}")
print(f"possible resolutions:\n\t{(NEWLINE + TAB).join([str(c) for c in db['outcome'].drop_duplicates()])}")
print(f"request types are: {(NEWLINE + TAB).join([str(c) for c in db['request_type'].drop_duplicates()])}")
print(f"mandatory fields per request type:")
db[['request_type', 'mandatory_fields']].drop_duplicates().sort_values(['mandatory_fields'])

db columns are:
	ticket_id
	created_at
	requester_department
	requester_title
	request_type
	request_summary
	details
	mandatory_fields
	fields_provided
	outcome
	security_risk_score
	resolution_time_hours
	approver_role
possible resolutions:
	Approved
	Info Requested
	Rejected
request types are: Permission Change
	Vendor Approval
	Network Access
	Firewall Change
	Data Export
	Cloud Resource Access
	DevTool Install
mandatory fields per request type:


,request_type,mandatory_fields
8,Cloud Resource Access,Business Justification; Data Sensitivity Level
0,Permission Change,Business Justification; Duration; Manager Appr...
7,Data Export,Business Justification; PII Involved; Data Des...
3,Network Access,Business Justification; Source CIDR; Approval ...
4,Firewall Change,Business Justification; Source System; Destina...
14,DevTool Install,Business Justification; Team Lead Approval
1,Vendor Approval,Vendor Security Questionnaire; Data Classifica...


So we have only 7 unique request types with well-defined mandatory fields.
we can start by defining a json structure for each, trying to fill it and see what happens.

it seems like there are some important non-mandatory fields, like the specific premissions and buckets requested in cloud resources access and permissions change, but I'll get to that later.

In [16]:
for group, subsection in db.groupby('request_type'):
    print(
        f"{group} security risk is: ({subsection['security_risk_score'].mean(), subsection['security_risk_score'].std()})"
        )
print('\n')
for group, subsection in db.groupby('outcome'):
    print(
        f"{group} security risk is: ({subsection['security_risk_score'].mean(), subsection['security_risk_score'].std()})"
        )

Cloud Resource Access security risk is: ((67.86764705882354, 9.876878216903702))
Data Export security risk is: ((73.15, 9.132064625075031))
DevTool Install security risk is: ((50.03731343283582, 11.184311458613662))
Firewall Change security risk is: ((78.81645569620254, 8.096424159663567))
Network Access security risk is: ((75.67114093959732, 8.559365783069456))
Permission Change security risk is: ((83.15753424657534, 7.165711800112209))
Vendor Approval security risk is: ((60.32846715328467, 12.303221415219994))


Approved security risk is: ((69.74629324546952, 14.318681349105843))
Info Requested security risk is: ((70.34666666666666, 13.732249926979957))
Rejected security risk is: ((88.55555555555556, 2.202197357780265))


In [17]:
db[db['outcome'] == 'Rejected'][['details']]

,details
257,Requesting temporary firewall rule to allow ou...
405,Requesting AdministratorAccess role for AWS ac...
424,Requesting temporary firewall rule to allow ou...
440,Request to allow MySQL traffic from internal s...
452,Requesting temporary firewall rule to allow ou...
476,Requesting temporary firewall rule to allow ou...
520,Requesting temporary firewall rule to allow ou...
569,Request to allow MySQL traffic from internal s...
570,Requesting temporary firewall rule to allow ou...
642,Requesting AdministratorAccess role for AWS ac...


In [18]:
db[(db['request_type'] == 'Data Export')][['details', 'mandatory_fields', 'security_risk_score']]

,details,mandatory_fields,security_risk_score
7,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,70
10,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,66
13,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,87
39,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,70
43,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,65
...,...,...,...
957,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,61
961,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,61
968,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,64
978,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,64


In [22]:
passing_scores = db[db['outcome'] == 'Approved']['security_risk_score']
failing_scores = db[db['outcome'] == 'Rejected']['security_risk_score']
print(f"overall a higher score is bad.\npassing scores are within: ({passing_scores.min()}, {passing_scores.max()})")
print(f"and failing scores are within: ({failing_scores.min()}, {failing_scores.max()})")

overall a higher score is bad.
passing scores are within: (30, 95)
and failing scores are within: (85, 92)


It seems like there's no real relevance to the security risk, so I'll just wing it and move on. if a request can be approved with a score of 95 and rejected with a score of 85, it means that everything between these values is up for personal grudges.

In [23]:

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

# model_name = "google/gemma-2-9b-it"
model_name = "mlfoundations-dev/S1_4o_mini"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
)
model.eval()

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))

Error importing huggingface_hub.hf_api: No module named 'requests.exceptions'; 'requests' is not a package


ModuleNotFoundError: No module named 'requests.exceptions'; 'requests' is not a package